# Рекомендательная модель LightFM

In [195]:
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
import numpy as np
import warnings
from collections import Counter

warnings.filterwarnings("ignore")

In [196]:
def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

In [197]:
train_data = pd.read_csv('df_train.csv', delimiter=';')
test_data = pd.read_csv('df_test.csv', delimiter=';')

Достаю все нужные MCC для каждого юзера в отдельные листы, а так же нахожу все уникальные MCC-коды:

In [198]:
user_ids = train_data['Id']
mcc_lists = train_data['Data'].apply(lambda x: list(map(int, x.split(','))))
mcc_lists_train_target = train_data['Target'].apply(lambda x: list(map(int, x.split(','))))
test_mcc_lists = test_data['Data'].apply(lambda x: list(map(int, x.split(','))))

unique_mccs = list(set(mcc for mcc_list in mcc_lists for mcc in mcc_list))
unique_mccs.sort()

Создаю словари для шифрования и обратного шифрования кодов:

In [199]:
mcc_to_id = {value: index for index, value in enumerate(unique_mccs)}
id_to_mcc = {index: value for index, value in enumerate(unique_mccs)}

Тут, для каждого пользователя, я создаю словарь, в котором будет храниться информация о том, сколько раз каждый уникальный MCC-код встретился в истории покупок пользователя. Все эти словари хранятся в одном большом словаре. Коды храняться в виде шифров. Все это нужно для правильного заполнения разреженной матрицы для последующего обучения модели LightFM
 

In [200]:
dicts = []
for user in user_ids:
    dictionary = {}
    for el in mcc_lists[user]:
        if mcc_to_id[el] in dictionary:
            dictionary[mcc_to_id[el]] += 1
        else:
            dictionary[mcc_to_id[el]] = 1
    dicts.append(dictionary)

Привожу данные к нужному для модели формату и определяю названия рядов и колонок таблицы:

In [201]:
interactions_data = []
for user_id, user_dict in enumerate(dicts):
    for mcc_id, count in user_dict.items():
        interactions_data.append((user_id, mcc_id, count))

df_column = [y for x, y in mcc_to_id.items()]
df_row = train_data['Id']

На данном этапе строится разреженная таблица, в которой ряды, это id пользователей, а колонки, это все уникальные зашифрованные MCC-коды. Веса - это количество встреченных MCC-кодов какого-то типа у какого-то пользователя:

In [202]:
dataset = Dataset()
dataset.fit(set(df_row), set(df_column))
(interactions, weights) = dataset.build_interactions(interactions_data)

Этап обучения модели. Я выбрал функию потерь WARP, так как она лучше других работает на задачах ранжирования. Нашу задачу можно интерпретировать как "Следущие 10 рекомендуемых покупок, где первая самая вероятная".
Экспериментировал с количеством эпох, и пришел к выводу, что лучше всего 10:

In [203]:
model = LightFM(loss='warp')
model.fit(interactions, sample_weight=weights,epochs=10, num_threads=4, verbose=True)

Epoch: 100%|██████████| 10/10 [00:01<00:00,  7.06it/s]


Предсказываем и собираем все предсказания в один массив для проверки mapk. 

In [204]:
all_predicted = []
for user_to_predict in user_ids:
    # уникальные значения для предсказания у текущего пользователя
    vals_to_predict = list(set([mcc_to_id[x] for x in mcc_lists[user_to_predict]]))
    user_ids_test = [user_to_predict]

    predictions = model.predict(user_to_predict, vals_to_predict)
    
    #LightFM возвращает отрицательные значения весов (в этой библиотеке)
    predicted_ids = np.argsort(-predictions)
    predicted_mcc_codes = [id_to_mcc[vals_to_predict[x]] for x in predicted_ids]
    # Нахожу самый часто встречаемый MCC у пользователя
    counter = Counter(vals_to_predict)
    most_popular_mcc_value = counter.most_common(1)[0][0]
    # Если предсказанных значений меньше 10, то добавляем самое популярное значение
    while len(predicted_mcc_codes) <= 10:
        predicted_mcc_codes.append(most_popular_mcc_value)
    all_predicted.append(predicted_mcc_codes[:10])

In [206]:
print(mapk(mcc_lists_train_target, all_predicted))

0.290096777324134
